In [22]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)
import sqlite3
import re
import random   


In [23]:



# Connect to the SQLite database
connection = sqlite3.connect('dialects_database.db')


cursor = connection.cursor()

# Execute a query to retrieve the names of all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# tables[0][0] == > id_text
# tables[1][0] == > id_dialect

# # Execute a query and read it into a DataFrame
id_text = pd.read_sql_query("SELECT * FROM id_text", connection)
id_dialect = pd.read_sql_query("SELECT * FROM id_dialect", connection)


# close the connection  
connection.close()

# for the main data frame   
raw_df=pd.merge(id_text, id_dialect, on='id', how='left')

In [24]:
raw_df

,id,text,dialect
0,1009754958479151232,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.,LY
1,1009794751548313600,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة,LY
2,1019989115490787200,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند,LY
3,1035479791758135168,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلاً. ثانياً شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات؟ يعني اليتيمة متستحقش تتزوج؟ وثالثاً ليش البنت هي بس لازم ادير الف حساب للراجل؟ هي متستحقش يندارلها الف حساب ولا هي عبدة؟,LY
4,1035481122921164800,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتيمة ولا بنت معندهش خوت. هدي اعصابك وفكينا من التخلف امتاعك,LY
...,...,...,...
147720,982775354405376000,الناس دي بتنفخ في قربة مقدودة بالدارجي كده البلد دي لو ما الكيزان ديل راحو مافي حياه شريفه في البلد دي لان سبب الفساد والمصائب الموجودة #التطلع_لسودان_افضل,SD
147721,980728852224860160,@Wail_Hamza @amiromer انت عايش وين بره السودان ولا شنو ماشايف البحصل دا,SD
147722,975502734668500992,مااحرم نفسي ميسي حريف ولعاب برضو ..\nمدريدي وافتخر 🇪🇸,SD
147723,980207908641263616,ياخي ديل ماخلو للشيطان وإبليس شي لكن يروحو وين من ربنا ونحن وراك وراك ياكوز ياعفن الأيام معدوده خلاص #تعرف_علي_الكيزان,SD


In [25]:
#EG => Egypt
#LY => Libya
#LB => Lebanon
#SD => Sudan
#MA => Morocco

In [26]:
count_st=pd.DataFrame(raw_df.dialect.value_counts().values, index=raw_df.dialect.value_counts().index, columns=['count'])

count_st['ratio']=count_st['count']/sum(count_st['count'])
count_st


,count,ratio
dialect,,
EG,57636,0.390157
LY,36499,0.247074
LB,27617,0.186949
SD,14434,0.097709
MA,11539,0.078111


In [27]:
class ArabicCleaning():
    def __init__(self):
        pass

    def clean(self, text):
        arabic_pattern =  r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF]'
        arabic_pattern_others =  r'[^\w\s\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF]'

        def remove_special_words(text):
            words = text.split()
            text = [word for word in words if '#' not in word and '_' not in word]
            text = ' '.join(text)
            return text
        
        def keep_only_arabic_letters(text):
            words = text.split()
            processed_words = []
            for word in words:
                arabic_letters_only = ''.join([char for char in word if (re.match(arabic_pattern, char) and (char not in ["؟", "؛" , "،"]))])
                processed_words.append(arabic_letters_only)
            return ' '.join(processed_words)
        
        def check_empty(text):
            if len(text.split())==0:
                return None
            else :
                return text
            
        text = remove_special_words(text)
        text = re.sub(arabic_pattern_others, '', text)
        text = re.sub(r'[0-9]', '', text)
        text = re.sub(r'[a-zA-Z]', '', text)
        text = keep_only_arabic_letters(text)
        text = check_empty(text)    

        return text


In [28]:
raw_df

,id,text,dialect
0,1009754958479151232,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.,LY
1,1009794751548313600,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة,LY
2,1019989115490787200,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند,LY
3,1035479791758135168,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلاً. ثانياً شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات؟ يعني اليتيمة متستحقش تتزوج؟ وثالثاً ليش البنت هي بس لازم ادير الف حساب للراجل؟ هي متستحقش يندارلها الف حساب ولا هي عبدة؟,LY
4,1035481122921164800,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتيمة ولا بنت معندهش خوت. هدي اعصابك وفكينا من التخلف امتاعك,LY
...,...,...,...
147720,982775354405376000,الناس دي بتنفخ في قربة مقدودة بالدارجي كده البلد دي لو ما الكيزان ديل راحو مافي حياه شريفه في البلد دي لان سبب الفساد والمصائب الموجودة #التطلع_لسودان_افضل,SD
147721,980728852224860160,@Wail_Hamza @amiromer انت عايش وين بره السودان ولا شنو ماشايف البحصل دا,SD
147722,975502734668500992,مااحرم نفسي ميسي حريف ولعاب برضو ..\nمدريدي وافتخر 🇪🇸,SD
147723,980207908641263616,ياخي ديل ماخلو للشيطان وإبليس شي لكن يروحو وين من ربنا ونحن وراك وراك ياكوز ياعفن الأيام معدوده خلاص #تعرف_علي_الكيزان,SD


In [29]:
df = raw_df.copy()
df['text'] = df['text'].apply(lambda x: ArabicCleaning().clean(x))


In [30]:
df

,id,text,dialect
0,1009754958479151232,قليلين ادب ومنافقين لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم,LY
1,1009794751548313600,الليبيين متقلبين بس بالنسبة ليا انا ميليشياوي زمان وتوة,LY
2,1019989115490787200,كل تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب ذي اند,LY
3,1035479791758135168,رانيا عقليتك متخلفة اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلاً ثانياً شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات يعني اليتيمة متستحقش تتزوج وثالثاً ليش البنت هي بس لازم ادير الف حساب للراجل هي متستحقش يندارلها الف حساب ولا هي عبدة,LY
4,1035481122921164800,شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتيمة ولا بنت معندهش خوت هدي اعصابك وفكينا من التخلف امتاعك,LY
...,...,...,...
147720,982775354405376000,الناس دي بتنفخ في قربة مقدودة بالدارجي كده البلد دي لو ما الكيزان ديل راحو مافي حياه شريفه في البلد دي لان سبب الفساد والمصائب الموجودة,SD
147721,980728852224860160,انت عايش وين بره السودان ولا شنو ماشايف البحصل دا,SD
147722,975502734668500992,مااحرم نفسي ميسي حريف ولعاب برضو مدريدي وافتخر,SD
147723,980207908641263616,ياخي ديل ماخلو للشيطان وإبليس شي لكن يروحو وين من ربنا ونحن وراك وراك ياكوز ياعفن الأيام معدوده خلاص,SD


In [31]:
for i in random.sample(range(0, sum(df.dialect=='EG')),10 ):
    print(raw_df[raw_df.dialect=='EG'].reset_index(drop=True).iloc[i]['text'])  
    print(df[df.dialect=='EG'].reset_index(drop=True).iloc[i]['text'],"\n")   


هع هع هع
إلبـــــــــــــــــس😂😂😂
*بصوت الواد الرخم في فيلم عمر و سلمى* https://t.co/KvJSCmeFVV
هع هع هع إلبـــــــــــــــــس بصوت الواد الرخم في فيلم عمر و سلمى 

#جمهور_الاهلي_وراك_ياخطيب
أنت الأسطورة وراك ملايين يا بيبو
أنت الأسطورة وراك ملايين يا بيبو 

#وايل_غنيم
واحد كان على قمة ثورة يناير
وقمة اهتمام الشباب والاعلام
وفجأة اتركن عى جنب سنين
وعايز يرجع تانى بؤرة اهتمام الشباب وخاصة اللى كانوا اطفال ايام الثورة ودلوقتى كبروا
يعمل ايه؟؟؟
يخرج عن المألوف عشان نسيب كل حاجة ونتابعه هو
وبعد كده يفرد ورقه اللى عايزنا نشوفه
بس خلاص🏃🏃 https://t.co/JlcQJ5smvg
واحد كان على قمة ثورة يناير وقمة اهتمام الشباب والاعلام وفجأة اتركن عى جنب سنين وعايز يرجع تانى بؤرة اهتمام الشباب وخاصة اللى كانوا اطفال ايام الثورة ودلوقتى كبروا يعمل ايه يخرج عن المألوف عشان نسيب كل حاجة ونتابعه هو وبعد كده يفرد ورقه اللى عايزنا نشوفه بس خلاص 

هي الناس اللي لسه اصدقاء لسميح ده إحساسهم ايه بعد الهبل والخيانة وعكه ده 🤔
هي الناس اللي لسه اصدقاء لسميح ده إحساسهم ايه بعد الهبل والخيانة وعكه ده 

@AlAhly اهو اللي مشي بر

In [32]:

# find the duplicated rows  
print("the number of duplicated rows in the raw data are: ", raw_df.text.duplicated().sum()) 
print("the nunmber of null values in the raw data are: ", df.text.isnull().sum())   
print("the number of duplicated rows in the cleaned data are: ", df.text.duplicated().sum()) 

the number of duplicated rows in the raw data are:  0
the nunmber of null values in the raw data are:  183
the number of duplicated rows in the cleaned data are:  252


In [33]:
df.dropna(subset=['text'], inplace=True)   
print("the number of duplicated rows in the cleaned data are: ", df[["text","dialect"]].duplicated().sum())  

the number of duplicated rows in the cleaned data are:  58


In [40]:
# show the duplicated rows  

df[df[["text"]].duplicated(keep=False)].sort_values('text').shape


(129, 3)

In [41]:
df.drop_duplicates(subset=["text"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [42]:
print("the number of duplicated rows in the cleaned data are: ", df[["text"]].duplicated().sum())  

the number of duplicated rows in the cleaned data are:  0


In [ ]:
save to csv 
df.to_csv('cleaned_data.csv', index=False)